# Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf
import statsmodels.api as sm
import matplotlib.pyplot as plt
import getFamaFrenchFactors as gff
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os

# Importing Dataset
Here, Just importing the monthly data from the getfamaFrenchFactors as this project focuses on checking the investment returns, through monthly investment with strategic seasonal investment.

In [3]:
ff3_monthly = gff.famaFrench3Factor(frequency='m')

Since, wanted to project the investment growth for the period of Jan 2010 to Dec 2019, filtering out that period from the whole dataset.

In [4]:
ff3_monthly = ff3_monthly[(ff3_monthly['date_ff_factors'] >= '2010-01') & (ff3_monthly['date_ff_factors'] < '2020-01')].reset_index(drop = True)
ff3_monthly

,date_ff_factors,Mkt-RF,SMB,HML,RF
0,2010-01-31,-0.0336,0.0040,0.0043,0.0000
1,2010-02-28,0.0340,0.0119,0.0323,0.0000
2,2010-03-31,0.0631,0.0148,0.0221,0.0001
3,2010-04-30,0.0200,0.0487,0.0289,0.0001
4,2010-05-31,-0.0789,0.0009,-0.0244,0.0001
...,...,...,...,...,...
115,2019-08-31,-0.0258,-0.0238,-0.0478,0.0016
116,2019-09-30,0.0143,-0.0096,0.0675,0.0018
117,2019-10-31,0.0206,0.0029,-0.0191,0.0016
118,2019-11-30,0.0387,0.0077,-0.0202,0.0012
